# \# PV Optimization Monitoring

This notebook provides a workflow for monitoring and optimizing photovoltaic (PV) systems within a microgrid. It guides the user through selecting the correct component types for a specific site, fetching and visualizing power flow and battery data, and analyzing monthly aggregates. 

In [1]:
import os
from datetime import datetime, timedelta

from frequenz.lib.notebooks.reporting.utils.helpers import set_date_to_midnight
from frequenz.gridpool import MicrogridConfig

import frequenz.lib.notebooks.reporting.asset_optimization.data as aodata
from frequenz.lib.notebooks.reporting.asset_optimization.viz_mpl import plot_monthly, plot_power_flow, plot_energy_trade, plot_power_flow_trade, plot_battery_power

In [17]:
mdata = aodata.init_microgrid_data(
    dotenv_path="./.env", 
    microgrid_config_dir="toml_directory",
)

directory = "toml_directory/"
configs = MicrogridConfig.load_configs(microgrid_config_dir=directory)
available_microgrids = sorted(list(int(x) for x in configs.keys()))

# Power flow

In [3]:
microgrid_id = '13'

In [4]:
component_types = mdata.microgrid_configs[str(microgrid_id)].component_types()
component_types

['grid', 'pv', 'chp', 'battery', 'consumption']

In [5]:

from dateutil.parser import parse as _deepnote_parse
end_date = _deepnote_parse('2026-02-10T00:00:00.000Z').date()


In [6]:
days_back = '7'

In [7]:
resolution = '900'

In [8]:
start_date = end_date - timedelta(days=int(days_back))

start_date = set_date_to_midnight(start_date, "Europe/Berlin")
end_date = set_date_to_midnight(end_date, "Europe/Berlin")

data = await aodata.fetch_data(
    mdata=mdata,
    component_types=component_types,
    mid=int(microgrid_id),
    start_time=start_date,
    end_time=end_date,
    resampling_period=timedelta(seconds=int(resolution)), 
    fetch_soc=True,
)

df = data.copy()

In [20]:
plot_power_flow_trade(df)

In [23]:
plot_battery_power(df)

# \# Monthly aggregates

In [11]:

from dateutil.parser import parse as _deepnote_parse
start_date_2 = _deepnote_parse('2026-01-01T00:00:00.000Z').date()


In [26]:
data2 = await aodata.fetch_data(
    mdata=mdata,
    component_types=component_types,
    mid=int(microgrid_id),
    start_time=start_date,
    end_time=end_date,  
    resampling_period=timedelta(seconds=int(resolution)),
    splits=True,
    fetch_soc=False,
)

df2 = data2
df2

In [29]:
months = plot_monthly(df2)
months

In [14]:
# Write the monthly data to a CSV file
# months.to_csv("monthly.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=800fa629-2132-42f8-a070-be1844de6378' target="_blank">

Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>